# Error Analysis

In this code, we generate samples of various sizes and learn errors versus sample size.

In [ ]:
using AdvancedHMC, ForwardDiff
using LogDensityProblems, LinearAlgebra, Random, Statistics, JuMP, Ipopt, Plots, Optim, GalacticOptim

# Define the SineGordonModel2D structure for the 2D model
struct SineGordonModel2D
    lattice_size::Int  # Lattice size (NxN)
    a::Float64         # Coupling strength for the sine term
    b::Float64        # Mass parameter squared (m^2)
end

# Function to calculate the total energy (kinetic + potential) of a 2D field configuration
function calculate_energy_2d(phi::Matrix{T}, model::SineGordonModel2D) where T<:Real
    nx, nt = size(phi)  # Dimensions of the lattice
    # Initialize energy terms using the type of `phi`
    kinetic_energy = zero(T)
    gradient_energy = zero(T)
    sine_energy = zero(T)

    # Loop through all lattice sites and calculate each term in the Hamiltonian
    for x in 1:nx
        for t in 1:nt
            # Calculate the temporal derivative term using discrete differences
            t_prev = mod1(t - 1, nt)  # Time neighbor with periodic boundary conditions
            kinetic_energy += T(0.5) * ((phi[x, t] - phi[x, t_prev]))^2

            # Calculate the spatial gradient term using discrete differences
            x_prev = mod1(x - 1, nx)  # Spatial neighbor with periodic boundary conditions
            gradient_energy += T(0.5) * ((phi[x, t] - phi[x_prev, t]))^2

            # Corrected sine interaction term
            sine_energy += (model.a / model.b^2) * (1 - cos(model.b * phi[x, t]))
        end
    end

    # Return the total energy
    total_energy = (kinetic_energy + gradient_energy + sine_energy)
    return total_energy
end


# Define the SineGordonModelDensity2D structure for 2D
struct SineGordonModelDensity2D
    dim::Int
    model::SineGordonModel2D
end

# Implement the log-density function for SineGordonModelDensity2D
function LogDensityProblems.logdensity(p::SineGordonModelDensity2D, phi_vector::Vector{T}) where T<:Real
    # Reshape the vector back into a 2D matrix
    N = p.model.lattice_size
    phi = reshape(phi_vector, N, N)
    return -calculate_energy_2d(phi, p.model)
end

LogDensityProblems.dimension(p::SineGordonModelDensity2D) = p.model.lattice_size^2
LogDensityProblems.capabilities(::Type{SineGordonModelDensity2D}) = LogDensityProblems.LogDensityOrder{1}()

# Function to generate data for the model using Hamiltonian Monte Carlo (HMC)
function generate_SG_data_hmc_2d(model::SineGordonModel2D, n_samples::Int, n_adapts::Int = 5000)
    lattice_size = model.lattice_size
    sg_target = SineGordonModelDensity2D(lattice_size^2, model)

    # Define the initial 2D field configuration (NxN matrix)
    initial_phi = randn(lattice_size, lattice_size)

    # Define a Hamiltonian system
    metric = DiagEuclideanMetric(lattice_size^2)
    hamiltonian = Hamiltonian(metric, sg_target, ForwardDiff)

    # Define a leapfrog solver with an initial step size chosen heuristically
    initial_epsilon = find_good_stepsize(hamiltonian, vec(initial_phi))
    integrator = Leapfrog(initial_epsilon)

    # Define an HMC sampler with Multinomial sampling and Generalized No-U-Turn
    kernel = HMCKernel(Trajectory{MultinomialTS}(integrator, GeneralisedNoUTurn()))
    adaptor = StanHMCAdaptor(MassMatrixAdaptor(metric), StepSizeAdaptor(0.8, integrator))

    # Run the sampler to draw samples from the model
    samples_vector, _ = sample(hamiltonian, kernel, vec(initial_phi), n_samples, adaptor, n_adapts; progress=false)

    # Reshape the samples back into 2D matrices
    samples = [reshape(sample, lattice_size, lattice_size) for sample in samples_vector]

    return samples
end

# Score matching objective function for 2D Sine-Gordon model
function score_matching_objective_2d(a::T, b::T, data::Vector{Matrix{Float64}}) where T<:Real
    nx, nt = size(data[1])
    n_samples = length(data)
    obj = zero(T)
    for sample in data
        for x in 1:nx
            for t in 1:nt
                phi_xt = sample[x, t]

                # Nearest neighbors with periodic boundary conditions
                t_prev = mod1(t - 1, nt)
                x_prev = mod1(x - 1, nx)
                t_next = mod1(t + 1, nt)
                x_next = mod1(x + 1, nx)
                
                # Sum of neighboring phi values
                sum_neighbor_phi = sample[x, t_prev] + sample[x, t_next] + sample[x_prev, t] + sample[x_next, t]

                # Corrected score function s(phi_xt)
                score_xt = -((4 * phi_xt - sum_neighbor_phi) + (a / b) * sin(b * phi_xt))

                # Corrected second derivative s'(phi_xt)
                s_prime = - (4 + a * cos(b * phi_xt))

                # Add to the objective
                obj += T(0.5) * score_xt^2 + s_prime
            end
        end
    end

    # Normalize the objective by the number of samples
    return (obj / n_samples)
end

# Function to estimate the parameters using score matching in 2D with m^2
using GalacticOptim, Optim  # both packages loaded

function estimate_parameters_score_matching_2d(data::Vector{Matrix{Float64}})
   # Define the objective function taking x = [a, b]
   f(x) = score_matching_objective_2d(x[1], x[2], data)

   # Set an initial guess and bounds (assume a and b are positive)
   initial_x = [0.1, 2.5]
   lower = [0.001, 0.0]
   upper = [25.0, 10.0]

   # Create an instance of ParticleSwarm with the desired options
   ps = ParticleSwarm(lower=lower, upper=upper)

   # Run the global optimizer using GalacticOptim.optimize
   result = optimize(f, initial_x, ps)

   # Extract the minimizer and the minimum objective value
   x_opt = result.minimizer
   f_opt = result.minimum

   println("Final objective value: ", f_opt)
   println("Estimated parameters: a = ", x_opt[1], ", b = ", x_opt[2])

   return x_opt[1], x_opt[2]
end

# Main script to run the experiments
Random.seed!(23926)

initial_lattice_size = 4
initial_alpha = 0.5
initial_beta = 0.9
initial_model = SineGordonModel2D(initial_lattice_size, initial_alpha, initial_beta)

true_alpha = initial_alpha
true_beta = initial_beta

# Sample sizes: 4000, 8000, ..., 1024000
sample_sizes = [4000 * 2^i for i in 2:8]
alpha_errors = Float64[]
beta_errors = Float64[]

for ns in sample_sizes
    rep_alpha = Float64[]
    rep_beta = Float64[]
    println("\nSample size: $ns")
    for rep in 1:10
        samples = generate_SG_data_hmc_2d(initial_model, ns)
        est_alpha, est_beta = estimate_parameters_score_matching_2d(samples)
        push!(rep_alpha, abs(est_alpha - true_alpha))
        push!(rep_beta, abs(est_beta - true_beta))
        println("  rep $rep: alpha = $est_alpha, beta = $est_beta")
    end
    push!(alpha_errors, mean(rep_alpha))
    push!(beta_errors, mean(rep_beta))
    println("Mean absolute error for $ns: alpha = $(mean(rep_alpha)), beta = $(mean(rep_beta))")
end